In [ ]:
# 1. Importer les bibliothèques nécessaires
import pandas as pd
import json
import folium
from shapely.geometry import Polygon
import os

os.makedirs('../generated/sites', exist_ok=True)
source_dir = '../generated/'
faits = pd.read_csv(f"{source_dir}faits_matieres.csv", sep=',', encoding='utf-8')
dim_type_matiere = pd.read_csv(f"{source_dir}dim_type_matiere.csv", sep=',', encoding='utf-8')
dim_producteur = pd.read_csv(f"{source_dir}dim_producteur.csv", sep=',', encoding='utf-8')
dim_temps = pd.read_csv(f"{source_dir}dim_temps.csv", sep=',', encoding='utf-8')
dim_site_valorisation = pd.read_csv(f"{source_dir}dim_site_valorisation.csv", sep=',', encoding='utf-8')

geojson_path = "../generated/normalized/collecte.json"

# Visualisations géospatiales croisées avancées
Ce qui suit présente plusieurs exemples de visualisations géospatiales croisées :
- Carte choroplèthe (volume par secteur)
- Heatmap de points (volume ou fréquence)
- Cluster de markers (densité)
- Animation temporelle (évolution par période)
- Flux de matières (producteur → site)
- Densité par type de matière
- Points chauds (hotspots)

# Centres de tri


In [ ]:
# 2bis. Afficher les centres de tri sur la carte avec leurs coordonnées GPS

import pandas as pd
import folium

# Charger les centres de tri géocodés
centres_path = '../generated/normalized/liste-centres-tri-crd.csv'
df_centres = pd.read_csv(centres_path)

# Créer la carte centrée sur le Québec
m_centres = folium.Map(location=[52, -71], zoom_start=5)

# Ajouter le geojson des secteurs (si besoin)
geojson_path = "../generated/normalized/collecte.json"
folium.GeoJson(geojson_path, name="Secteurs collecte").add_to(m_centres)

# Ajouter un marker pour chaque centre de tri avec coordonnées GPS valides
for _, row in df_centres.iterrows():
    try:
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        if pd.notnull(lat) and pd.notnull(lon):
            folium.Marker(
                location=[lat, lon],
                popup=f"{row.get('Nom', 'Centre de tri')}\n{row.get('Adresse ', '')}",
                icon=folium.Icon(color='green', icon='recycle', prefix='fa')
            ).add_to(m_centres)
    except Exception:
        continue

m_centres.save('../generated/sites/centres_tri_map.html')
print("Carte interactive des centres de tri générée : centres_tri_map.html")

In [ ]:
# Cluster de markers : densité de faits à partir de faits_matieres.csv
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes nécessaires
for col in ['latitude', 'longitude', 'quantite_generee_donnees_agglo']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

m_cluster = folium.Map(location=[45.55, -73.6], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m_cluster)

for _, row in df.dropna(subset=['latitude', 'longitude', 'matiere', 'quantite_generee_donnees_agglo', 'territoire']).iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['matiere']} ({row['quantite_generee_donnees_agglo']} t) - {row['territoire']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

m_cluster.save('../generated/sites/cluster_markers.html')
print('Cluster de markers généré : cluster_markers.html')

In [ ]:
# Flux de matières : lignes entre producteurs (territoire) et leur site de valorisation le plus proche
import pandas as pd
import folium
from geopy.distance import geodesic

# Charger les faits matières et les centres de tri
faits = pd.read_csv('../generated/faits_matieres.csv')
centres = pd.read_csv('../generated/normalized/liste-centres-tri-crd.csv')

# Nettoyer les colonnes nécessaires
for col in ['latitude', 'longitude']:
    faits[col] = pd.to_numeric(faits[col], errors='coerce')
for col in ['latitude', 'longitude']:
    centres[col] = pd.to_numeric(centres[col], errors='coerce')

# Carte centrée sur Montréal
m_flux = folium.Map(location=[45.55, -73.6], zoom_start=10)

# Ajouter un marker pour chaque territoire (producteur)
for _, row in faits.dropna(subset=['latitude', 'longitude', 'territoire']).iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Producteur: {row['territoire']}",
        icon=folium.Icon(color='blue', icon='industry', prefix='fa')
    ).add_to(m_flux)

# Ajouter un marker pour chaque centre de tri (site de valorisation)
for _, row in centres.dropna(subset=['latitude', 'longitude']).iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Site de valorisation: {row.get('Nom', 'Centre de tri')}",
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(m_flux)

# Relier chaque producteur à son centre de tri le plus proche
for _, prod in faits.dropna(subset=['latitude', 'longitude', 'territoire']).iterrows():
    prod_loc = (prod['latitude'], prod['longitude'])
    min_dist = float('inf')
    closest_site = None
    for _, site in centres.dropna(subset=['latitude', 'longitude']).iterrows():
        site_loc = (site['latitude'], site['longitude'])
        dist = geodesic(prod_loc, site_loc).kilometers
        if dist < min_dist:
            min_dist = dist
            closest_site = site_loc
    if closest_site:
        folium.PolyLine(
            locations=[prod_loc, closest_site],
            color='purple', weight=2, opacity=0.6
        ).add_to(m_flux)

m_flux.save('../generated/sites/flux_matieres.html')
print('Carte des flux de matières générée : flux_matieres.html')

In [ ]:
# Densité par type de matière : markers colorés par type à partir de faits_matieres.csv
import pandas as pd
import folium

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes nécessaires
for col in ['latitude', 'longitude', 'quantite_generee_donnees_agglo']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

type_colors = {
    'Plastique': 'red',
    'Métal': 'blue',
    'Papier': 'green',
    'Verre': 'purple',
    'Organique': 'orange',
    # Ajouter d'autres types si nécessaire
}

m_density = folium.Map(location=[45.55, -73.6], zoom_start=10)

for _, row in df.dropna(subset=['latitude', 'longitude', 'matiere', 'quantite_generee_donnees_agglo']).iterrows():
    color = type_colors.get(row['matiere'], 'gray')
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['matiere']} ({row['quantite_generee_donnees_agglo']} t)",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(m_density)

m_density.save('../generated/sites/densite_type_matiere.html')
print('Carte densité par type de matière générée : densite_type_matiere.html')

In [ ]:
# Points chauds (hotspots) par volume à partir de faits_matieres.csv
import pandas as pd
import numpy as np
import folium

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes numériques
for col in ['quantite_generee_donnees_agglo', 'latitude', 'longitude']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Préparer les points avec volume (quantite_generee_donnees_agglo)
hot_points = df.dropna(subset=['latitude', 'longitude', 'quantite_generee_donnees_agglo'])
hot_points = hot_points[['latitude', 'longitude', 'quantite_generee_donnees_agglo']].values.tolist()

# Définir un seuil pour les hotspots (exemple : top 10 %)
volumes = np.array([p[2] for p in hot_points])
if len(volumes) > 0:
    seuil = np.percentile(volumes, 90)
    hotspots = [p for p in hot_points if p[2] >= seuil]

    m_hotspot = folium.Map(location=[45.55, -73.6], zoom_start=10)
    for pt in hotspots:
        folium.CircleMarker(
            location=[pt[0], pt[1]],
            radius=10,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=f'Hotspot: {pt[2]} t'
        ).add_to(m_hotspot)

    m_hotspot.save('../generated/sites/hotspots_volume.html')
    print('Carte des hotspots générée : hotspots_volume.html')
else:
    print('Aucun point valide pour générer des hotspots.')

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes numériques (au cas où il y aurait des virgules ou des valeurs manquantes)
for col in ['quantite_generee_donnees_agglo', 'quantite_collectee_donnees_agglo', 'latitude', 'longitude']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Carte centrée sur Montréal
m = folium.Map(location=[45.55, -73.6], zoom_start=10)

# Heatmap pour la quantité générée dans un FeatureGroup
fg_generee = folium.FeatureGroup(name='Quantité générée (agglo)')
heat_data_generee = df.dropna(subset=['latitude', 'longitude', 'quantite_generee_donnees_agglo'])
heat_generee = [
    [row['latitude'], row['longitude'], row['quantite_generee_donnees_agglo']]
    for _, row in heat_data_generee.iterrows()
]
HeatMap(heat_generee, min_opacity=0.4, radius=20, blur=15, max_zoom=1).add_to(fg_generee)
fg_generee.add_to(m)

# Heatmap pour la quantité collectée dans un FeatureGroup
fg_collectee = folium.FeatureGroup(name='Quantité collectée (agglo)')
heat_data_collectee = df.dropna(subset=['latitude', 'longitude', 'quantite_collectee_donnees_agglo'])
heat_collectee = [
    [row['latitude'], row['longitude'], row['quantite_collectee_donnees_agglo']]
    for _, row in heat_data_collectee.iterrows()
]
HeatMap(heat_collectee, min_opacity=0.4, radius=20, blur=15, max_zoom=1, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(fg_collectee)
fg_collectee.add_to(m)

folium.LayerControl().add_to(m)
m.save('../generated/sites/heatmap_faits_matieres.html')
print("Carte HeatMap générée : heatmap_faits_matieres.html")

## Carte choroplèthe : volume total par secteur (GeoJSON)

## Heatmap par type de matière (une heatmap par type, superposables)

In [ ]:
from folium.plugins import HeatMap

m_heat_types = folium.Map(location=[45.55, -73.6], zoom_start=10)
types = faits['matiere'].dropna().unique()

# S'assurer que la colonne est bien numérique
faits['quantite_generee_donnees_agglo'] = pd.to_numeric(faits['quantite_generee_donnees_agglo'], errors='coerce')

for mat in types:
    fg = folium.FeatureGroup(name=f'Heatmap {mat}')
    sub = faits[(faits['matiere'] == mat) & faits['latitude'].notnull() & faits['longitude'].notnull()]
    # On ne garde que les lignes où la quantité est numérique et non nulle
    sub = sub.dropna(subset=['quantite_generee_donnees_agglo'])
    heat_data = sub[['latitude', 'longitude', 'quantite_generee_donnees_agglo']].values.tolist()
    if heat_data:
        HeatMap(heat_data, min_opacity=0.4, radius=18, blur=12, max_zoom=1).add_to(fg)
        fg.add_to(m_heat_types)

folium.LayerControl().add_to(m_heat_types)
m_heat_types.save('../generated/sites/heatmap_types_matieres.html')
print('Heatmap par type de matière générée : heatmap_types_matieres.html')